Einfache SQL-Abfrage

In [0]:
%sql
Select "hallo" as msg

msg
hallo


## SQL-Table zu Python

Tabelle 1

In [0]:
fact = spark.sql(
    """SELECT * FROM (
        VALUES
            ('2025-09-11', 1),
            ('2025-09-11', 1),
            ('2025-09-11', 2),
            ('2025-09-12', 2),
            ('2025-09-12', 3),
            ('2025-09-12', 5),
            ('2025-09-13', 2)
    ) AS products_table(date, produkt_id);"""
)

fact.show()

+----------+----------+
|      date|produkt_id|
+----------+----------+
|2025-09-11|         1|
|2025-09-11|         1|
|2025-09-11|         2|
|2025-09-12|         2|
|2025-09-12|         3|
|2025-09-12|         5|
|2025-09-13|         2|
+----------+----------+



In [0]:
dates = fact.select("date")

dates.show()


+----------+
|      date|
+----------+
|2025-09-11|
|2025-09-11|
|2025-09-11|
|2025-09-12|
|2025-09-12|
|2025-09-12|
|2025-09-13|
+----------+



Tabelle 2

In [0]:
product = spark.sql(
    """WITH my_temp_table AS (
        SELECT * FROM (
            VALUES
                ('Produkt 1'),
                ('Produkt 2'),
                ('Produkt 3'),
                ('Produkt 4'),
                ('Produkt 5'),
                ('Produkt 6'),
                ('Produkt 7'),
                ('Produkt 8'),
                ('Produkt 9')
        ) AS a(msg)
    )
    SELECT 
        ROW_NUMBER() OVER(ORDER BY msg) as id,
        msg
    FROM my_temp_table;"""
)

product.show()

+---+---------+
| id|      msg|
+---+---------+
|  1|Produkt 1|
|  2|Produkt 2|
|  3|Produkt 3|
|  4|Produkt 4|
|  5|Produkt 5|
|  6|Produkt 6|
|  7|Produkt 7|
|  8|Produkt 8|
|  9|Produkt 9|
+---+---------+



Join

In [0]:
# Registriere die DataFrames als temporäre Ansichten
fact.createOrReplaceTempView("fact_table")
product.createOrReplaceTempView("product_table")

# Führe den JOIN über die temporären Ansichten aus
result = spark.sql(
    """SELECT
        t1.date,
        t1.produkt_id,
        t2.msg as produkt_name
    FROM
        fact_table t1
    JOIN
        product_table t2 ON t1.produkt_id = t2.id
    """
)

result.show()

+----------+----------+------------+
|      date|produkt_id|produkt_name|
+----------+----------+------------+
|2025-09-11|         1|   Produkt 1|
|2025-09-13|         2|   Produkt 2|
|2025-09-12|         3|   Produkt 3|
|2025-09-12|         5|   Produkt 5|
|2025-09-11|         1|   Produkt 1|
|2025-09-12|         2|   Produkt 2|
|2025-09-11|         2|   Produkt 2|
+----------+----------+------------+



Spark DataFrame in Pandas DataFrame konvertieren

In [0]:
# Konvertiert das Spark DataFrame in ein Pandas DataFrame
df_pandas = result.toPandas()

# Zeigt den DataFrame an (mit allen Spalten)
print(df_pandas)

         date  produkt_id produkt_name
0  2025-09-11           1    Produkt 1
1  2025-09-11           1    Produkt 1
2  2025-09-13           2    Produkt 2
3  2025-09-12           2    Produkt 2
4  2025-09-11           2    Produkt 2
5  2025-09-12           3    Produkt 3
6  2025-09-12           5    Produkt 5


In [0]:
# Greife auf Spalten wie bei einem Python Dictionary zu
print("\nZugriff auf die Spalte 'produkt_name':")
print(df_pandas['produkt_name'])


Zugriff auf die Spalte 'produkt_name':
0    Produkt 1
1    Produkt 1
2    Produkt 2
3    Produkt 2
4    Produkt 2
5    Produkt 3
6    Produkt 5
Name: produkt_name, dtype: object


In [0]:
# Zählt die Vorkommen jedes Produktnamens
product_counts = df_pandas['produkt_name'].value_counts()

# Zeigt das Ergebnis an
print(product_counts)

Produkt 2    3
Produkt 1    2
Produkt 3    1
Produkt 5    1
Name: produkt_name, dtype: int64
